In [1]:
from googleapiclient.discovery import build
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime

In [2]:
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")

if not API_KEY:
    raise ValueError("YouTube API key not found in .env file!")

In [3]:
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [4]:
def search_videos(query, max_results=20):
    """
    Search for videos by keyword.
    """
    response = youtube.search().list(
        q=query,
        part='id,snippet',
        maxResults=max_results,
        type='video'
    ).execute()

    videos = []
    for item in response['items']:
        snippet = item['snippet']
        videos.append({
            'video_id': item['id']['videoId'],
            'channel_id': snippet['channelId'],
            'title': snippet['title'],
            'description': snippet['description'],
            'published_at': snippet['publishedAt'],
            'channel_title': snippet['channelTitle'],
            'search_query': query
        })
    return pd.DataFrame(videos)

In [5]:
videos_df = search_videos("premier league", max_results=25)
videos_df.head()

,video_id,channel_id,title,description,published_at,channel_title,search_query
0,IuuzR-Cc-es,UCG5qGWdu8nIRZqJ_GgDwQ-w,Premier League Managers Settle Internet&#39;s ...,Join Premier League managers as they debate th...,2025-10-08T10:01:40Z,Premier League,premier league
1,2EjuPNlC9QM,UCqZQlzSHbVJrwrn5XvzrzcA,Brentford v. Manchester City | PREMIER LEAGUE ...,Take a look back on full-match highlights from...,2025-10-05T18:15:10Z,NBC Sports,premier league
2,Gbz3J8ud0VY,UCqZQlzSHbVJrwrn5XvzrzcA,Chelsea v. Liverpool | PREMIER LEAGUE HIGHLIGH...,Relive full-match highlights from Liverpool's ...,2025-10-04T19:26:48Z,NBC Sports,premier league
3,_KW8i87KgTM,UCqZQlzSHbVJrwrn5XvzrzcA,Manchester United v. Sunderland | PREMIER LEAG...,Look back on full-match highlights from Sunder...,2025-10-04T17:04:22Z,NBC Sports,premier league
4,qk6Oo_SMgt4,UCqZQlzSHbVJrwrn5XvzrzcA,Arsenal v. West Ham | PREMIER LEAGUE HIGHLIGHT...,Relive full-match highlights from Arsenal's Lo...,2025-10-04T16:45:35Z,NBC Sports,premier league


In [6]:
def get_channel_details(channel_ids):
    """
    Retrieve basic channel information.
    """
    response = youtube.channels().list(
        part="snippet,statistics",
        id=",".join(channel_ids)
    ).execute()

    channels = []
    for item in response['items']:
        snippet = item['snippet']
        stats = item['statistics']
        channels.append({
            'channel_id': item['id'],
            'channel_title': snippet['title'],
            'country': snippet.get('country'),
            'published_at': snippet['publishedAt'],
            'subscriber_count': int(stats.get('subscriberCount', 0)),
            'video_count': int(stats.get('videoCount', 0)),
            'view_count': int(stats.get('viewCount', 0))
        })
    return pd.DataFrame(channels)


In [7]:
channel_ids = videos_df['channel_id'].unique().tolist()
channels_df = get_channel_details(channel_ids)
channels_df.head()

,channel_id,channel_title,country,published_at,subscriber_count,video_count,view_count
0,UCtK4QAczAN2mt2ow_jlGinQ,Everton Football Club,GB,2011-09-23T22:22:17Z,934000,4319,342203296
1,UCjCJ2LaOIsPzOoXUTMDI3wg,Pitch Side,GB,2021-08-17T10:10:25.089826Z,332000,2049,214411955
2,UCqZQlzSHbVJrwrn5XvzrzcA,NBC Sports,US,2012-02-07T14:52:11Z,5080000,42221,4906709926
3,UCWB9N0012fG6bGyj486Qxmg,Crystal Palace FC,GB,2011-08-02T09:17:58Z,370000,3417,108238055
4,UC14BPCP-fxIDuG0I_dC2Kpg,FPL Tom,GB,2022-02-20T05:34:04.899656Z,19600,816,3697622


In [8]:
def get_video_stats(video_ids):
    """
    Retrieve engagement metrics for each video.
    """
    all_stats = []
    for i in range(0, len(video_ids), 50):  # API limit = 50 per call
        response = youtube.videos().list(
            part="statistics,snippet,contentDetails",
            id=",".join(video_ids[i:i+50])
        ).execute()

        for item in response['items']:
            stats = item['statistics']
            snippet = item['snippet']
            details = item['contentDetails']

            all_stats.append({
                'video_id': item['id'],
                'title': snippet['title'],
                'category_id': snippet.get('categoryId'),
                'duration': details.get('duration'),
                'views': int(stats.get('viewCount', 0)),
                'likes': int(stats.get('likeCount', 0)),
                'comments': int(stats.get('commentCount', 0)),
                'favorite_count': int(stats.get('favoriteCount', 0)),
                'collected_at': datetime.utcnow().isoformat()
            })
    return pd.DataFrame(all_stats)


In [9]:
video_ids = videos_df['video_id'].tolist()
video_stats_df = get_video_stats(video_ids)
video_stats_df.head()

C:\Users\adria\AppData\Local\Temp\ipykernel_69040\3760570532.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'collected_at': datetime.utcnow().isoformat()


,video_id,title,category_id,duration,views,likes,comments,favorite_count,collected_at
0,IuuzR-Cc-es,Premier League Managers Settle Internet's Bigg...,17,PT6M45S,33629,1466,77,0,2025-10-08T18:28:26.962379
1,2EjuPNlC9QM,Brentford v. Manchester City | PREMIER LEAGUE ...,17,PT8M44S,308829,3092,145,0,2025-10-08T18:28:26.962459
2,Gbz3J8ud0VY,Chelsea v. Liverpool | PREMIER LEAGUE HIGHLIGH...,17,PT13M12S,1149314,15429,790,0,2025-10-08T18:28:26.962477
3,_KW8i87KgTM,Manchester United v. Sunderland | PREMIER LEAG...,17,PT11M11S,592536,6787,307,0,2025-10-08T18:28:26.962488
4,qk6Oo_SMgt4,Arsenal v. West Ham | PREMIER LEAGUE HIGHLIGHT...,17,PT9M31S,407147,4490,177,0,2025-10-08T18:28:26.962498


We can only fetch up to 100 comments per video per API call, and the quota cost is 1 per video

In [10]:
def get_video_comments(video_id, max_comments=50):
    """
    Retrieve top-level comments for a given video.
    Returns a DataFrame with comment text, author, likes, and date.
    """
    comments = []
    next_page_token = None
    total_fetched = 0

    while total_fetched < max_comments:
        try:
            response = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(100, max_comments - total_fetched),
                pageToken=next_page_token,
                textFormat="plainText"
            ).execute()
        except Exception as e:
            print(f"❌ Error fetching comments for video {video_id}: {e}")
            break

        for item in response.get("items", []):
            snippet = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "video_id": video_id,
                "comment_id": item["id"],
                "author": snippet.get("authorDisplayName"),
                "text": snippet.get("textDisplay"),
                "like_count": snippet.get("likeCount", 0),
                "published_at": snippet.get("publishedAt"),
                "updated_at": snippet.get("updatedAt")
            })

        total_fetched += len(response.get("items", []))
        next_page_token = response.get("nextPageToken")

        if not next_page_token:
            break

    return pd.DataFrame(comments)

In [ ]:
# Sample of the first video only
sample_video = videos_df['video_id'].iloc[0]
comments_df = get_video_comments(sample_video, max_comments=30)
comments_df.head()

,video_id,comment_id,author,text,like_count,published_at,updated_at
0,IuuzR-Cc-es,UgxSh6cP561B4iYuUf14AaABAg,@premierleague,Do you agree with our managers' takes? What su...,39,2025-10-08T10:43:04Z,2025-10-08T10:43:04Z
1,IuuzR-Cc-es,Ugwrh1MrQ8Wg28fyg1x4AaABAg,@Deadpool1322,Someone investigate Marco Silva on the banana ...,0,2025-10-08T18:16:28Z,2025-10-08T18:16:28Z
2,IuuzR-Cc-es,UgzR8VJv4dyDqqjd6up4AaABAg,@vertigo10yearsago25,You could block off the bottom of the straw an...,0,2025-10-08T17:02:37Z,2025-10-08T17:02:37Z
3,IuuzR-Cc-es,UgxZ_Dcwf5ikBJLwpk94AaABAg,@AneleMgciniMdolomba007,Enzo Maresca's reactions and then nowhere to b...,1,2025-10-08T16:18:44Z,2025-10-08T16:18:44Z
4,IuuzR-Cc-es,Ugz5xVNoVokEewGUzxh4AaABAg,@tomstar1800,Good to see the more human side to all of thes...,0,2025-10-08T16:07:14Z,2025-10-08T16:07:14Z


In [ ]:
# 10 comments from the first 5 videos
all_comments = pd.concat([
    get_video_comments(v, max_comments=10)
    for v in videos_df['video_id'].head(5)
], ignore_index=True)

all_comments.head()

,video_id,comment_id,author,text,like_count,published_at,updated_at
0,IuuzR-Cc-es,UgxSh6cP561B4iYuUf14AaABAg,@premierleague,Do you agree with our managers' takes? What su...,39,2025-10-08T10:43:04Z,2025-10-08T10:43:04Z
1,IuuzR-Cc-es,Ugwrh1MrQ8Wg28fyg1x4AaABAg,@Deadpool1322,Someone investigate Marco Silva on the banana ...,0,2025-10-08T18:16:28Z,2025-10-08T18:16:28Z
2,IuuzR-Cc-es,UgzR8VJv4dyDqqjd6up4AaABAg,@vertigo10yearsago25,You could block off the bottom of the straw an...,0,2025-10-08T17:02:37Z,2025-10-08T17:02:37Z
3,IuuzR-Cc-es,UgxZ_Dcwf5ikBJLwpk94AaABAg,@AneleMgciniMdolomba007,Enzo Maresca's reactions and then nowhere to b...,2,2025-10-08T16:18:44Z,2025-10-08T16:18:44Z
4,IuuzR-Cc-es,Ugz5xVNoVokEewGUzxh4AaABAg,@tomstar1800,Good to see the more human side to all of thes...,0,2025-10-08T16:07:14Z,2025-10-08T16:07:14Z


In [16]:
def get_video_categories(region_code="US"):
    """
    Retrieve video categories for a specific region.
    Returns a DataFrame mapping category_id to category_name.
    """
    response = youtube.videoCategories().list(
        part="snippet",
        regionCode=region_code
    ).execute()

    categories = []
    for item in response["items"]:
        snippet = item["snippet"]
        categories.append({
            "category_id": item["id"],
            "category_title": snippet["title"],
            "assignable": snippet["assignable"],
            "region": region_code
        })
    return pd.DataFrame(categories)


In [17]:
categories_df = get_video_categories(region_code="US")
categories_df.head()

,category_id,category_title,assignable,region
0,1,Film & Animation,True,US
1,2,Autos & Vehicles,True,US
2,10,Music,True,US
3,15,Pets & Animals,True,US
4,17,Sports,True,US
